# Agenda

1. Data set that we're using -- NYC taxis
2. Load the data set and perform some basic operations with it
3. Use the filenames to set column information
4. Do some more analysis, comparing pandemic to non-pandemic
5. Be a bit smarter about reading + using data from the data set to set column information
6. Was this really smarter?

In [1]:
!ls *.csv

nyc_taxi_2019-01.csv  nyc_taxi_2020-01.csv
nyc_taxi_2019-07.csv  nyc_taxi_2020-07.csv


In [2]:
!ls -l *.csv

-rw-r--r-- 1 reuven staff 687088084 Sep 13 18:28 nyc_taxi_2019-01.csv
-rw-r--r-- 1 reuven staff 584387609 Sep 13 18:28 nyc_taxi_2019-07.csv
-rw-r--r-- 1 reuven staff 593610736 Sep 13 18:28 nyc_taxi_2020-01.csv
-rw-r--r-- 1 reuven staff  73326707 Sep 13 18:28 nyc_taxi_2020-07.csv


In [3]:
import pandas as pd
from pandas import Series, DataFrame

In [15]:
# to load CSV data into pandas, we can use pd.read_csv

import numpy as np

df = pd.read_csv('nyc_taxi_2019-01.csv',
                usecols=['passenger_count', 'trip_distance', 'total_amount',
                        'payment_type'],
                dtype={'passenger_count':np.int8,
                      'payment_type':np.int8,
                      'trip_distance':np.float16,
                      'total_amount':np.float16})

In [16]:
df.head() 

,passenger_count,trip_distance,payment_type,total_amount
0,1,1.500000,1,9.953125
1,1,2.599609,1,16.296875
2,3,0.000000,1,5.800781
3,5,0.000000,2,7.550781
4,5,0.000000,2,55.562500


In [17]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7667792 entries, 0 to 7667791
Data columns (total 4 columns):
 #   Column           Dtype  
---  ------           -----  
 0   passenger_count  int8   
 1   trip_distance    float16
 2   payment_type     int8   
 3   total_amount     float16
dtypes: float16(2), int8(2)
memory usage: 43.9 MB


In [26]:
import glob

all_dfs = []

for one_filename in glob.glob('nyc*-07.csv'):
    year = int(one_filename[9:13])

    mini_df = pd.read_csv(one_filename,
                    usecols=['passenger_count', 'trip_distance', 'total_amount',
                            'payment_type'])
    mini_df['year'] = year
    
    all_dfs.append(mini_df)
    
    
    
    

In [27]:
# now create one big data frame from our individual data frames

df = pd.concat(all_dfs)

In [28]:
df.head()

,passenger_count,trip_distance,payment_type,total_amount,year
0,1.0,1.50,2.0,9.30,2020
1,1.0,9.50,1.0,27.80,2020
2,1.0,5.85,2.0,22.30,2020
3,1.0,1.90,1.0,14.16,2020
4,1.0,1.25,2.0,7.80,2020


In [29]:
df.shape

(7110831, 5)

In [30]:
# how many taxi rides were there in July 2019 vs. July 2020?

df['year'].value_counts()

2019    6310419
2020     800412
Name: year, dtype: int64

In [31]:
# did people take taxis for less distance in 2020 than 2019?

# I could grab all rows from 2019, and get the mean distance
# and then grab all rows from 2020, and get the mean distance

In [32]:
df['year'] == 2019   # boolean series

0          False
1          False
2          False
3          False
4          False
           ...  
6310414     True
6310415     True
6310416     True
6310417     True
6310418     True
Name: year, Length: 7110831, dtype: bool

In [36]:
df.loc[df['year'] == 2019, 'trip_distance'].mean()

3.110131854952549

In [37]:
df.loc[df['year'] == 2020, 'trip_distance'].mean()

4.304164880086777

In [39]:
# I want to, with one query, get the mean trip_distance for each of these years
# we can do that with groupby

df.groupby('year').mean()

,passenger_count,trip_distance,payment_type,total_amount
year,,,,
2019,1.572045,3.110132,1.305580,19.612299
2020,1.378401,4.304165,1.352521,18.631460


In [40]:
df.groupby('year')[['passenger_count', 'trip_distance', 'total_amount']].mean()

,passenger_count,trip_distance,total_amount
year,,,
2019,1.572045,3.110132,19.612299
2020,1.378401,4.304165,18.631460


In [46]:
# what about payment types?

# my assumption is: people will use credit cards to pay, to avoid using cash and touching things
# that other people touched

# before the pandemic, most people paid with credit cards, but a good number paid with cash
df.loc[df['year'] == 2019, 'payment_type'].value_counts()

1.0    4424167
2.0    1801718
3.0      35485
4.0      15090
Name: payment_type, dtype: int64

In [47]:
df.loc[df['year'] == 2019, 'payment_type'].value_counts(normalize=True)

1.0    0.704883
2.0    0.287060
3.0    0.005654
4.0    0.002404
Name: payment_type, dtype: float64

In [48]:
df.loc[df['year'] == 2020, 'payment_type'].value_counts(normalize=True)

1.0    0.665705
2.0    0.320559
3.0    0.009245
4.0    0.004490
Name: payment_type, dtype: float64

In [ ]:
# couldn't I get the year from the data itself?  Why do I need to work so hard?

import glob

all_dfs = []

for one_filename in glob.glob('nyc*-07.csv'):
    year = int(one_filename[9:13])

    mini_df = pd.read_csv(one_filename,
                    usecols=['tpep_pickup_datetime',
                             'tpep_dropoff_datetime',
                             'passenger_count', 'trip_distance', 'total_amount',
                            'payment_type'])
    mini_df['year'] = year
    
    all_dfs.append(mini_df)
    
    
    
    